In [ ]:
## Libraries
# Data preprocessing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Figure
import matplotlib.pyplot as plt
import seaborn as sns

# Model
from sklearn.linear_model import LogisticRegression

# Path
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Refer to https://www.kaggle.com/mehrankazeminia/1-tps-jun-21-histgradient-catboost-nn

If you satisfied, you should upvote this link and my notebook :)

In [ ]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test_df  = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
sample_df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')


In [ ]:
# Read and check datasets.
train_df.shape,   test_df.shape
train_df.info(),  test_df.info()
train_df.head(5), train_df.tail(5)
test_df.head(5),  test_df.tail(5)

In [ ]:
# Ploting datasets which is presented.
# Only checking for dataframe of train.

# How about amount is dominant for columns using histogram
# To plot Bar and Circle chart
train_df['target'].value_counts().plot(kind='bar')


In [ ]:
train_df['target'].value_counts().plot(kind='pie')


Correlation for features

In [ ]:
# Define y value of dataframe
train_y = pd.DataFrame(columns=['target'])

# drop 'class' of target column and redefine to start 0 of class
train_df['target'] = train_df['target'].str.slice(start=6).astype(int)-1
train_y= train_df['target']

train_df.head(), train_y.head()

**Hot to select features?**

The origin of target valus is not numeric.

It's good to adopt features selection of using Mutual information (MI) than correlation.

**Caluclation of correlation**

In [ ]:
# Caculation of correlation
corrmat = train_df.corr()
limitValue = 0
top_corr_features = corrmat.index[abs(corrmat['target'])>limitValue]
top_corr_features

In [ ]:
# Heatmap
plt.figure(figsize=(13,10))
plt_corr = sns.heatmap(train_df[top_corr_features].corr(), annot=True)

**Mutual information(MI)**


In [ ]:
# Feature selection
train_x = train_df.drop(columns = ['id', 'target'])
test_x = test_df.drop(columns = ['id'])
train_x.head(), test_x.head()

In [ ]:
# Split train and val
from sklearn.model_selection import train_test_split
train_df_x, val_df_x, train_df_y, val_df_y = train_test_split(train_x, train_y, test_size=0.5, random_state=123)


In [ ]:
# Scaling and Split
from mlxtend.preprocessing import minmax_scaling
train_x_scaled = minmax_scaling(train_x, columns=train_x.columns)
train_df_x_scaled, val_df_s_scaled, train_df_y_scaled, val_df_y_scaled = train_test_split(train_x_scaled, train_y, test_size = 0.5, random_state=123)

test_x_scaled = minmax_scaling(test_x, columns=test_x.columns)


In [ ]:
# Features

train_x_feature = train_x.copy()
train_y_feature = train_y.copy()
for colname in train_x_feature.select_dtypes('object'):
    train_x_feature[colname], _ = train_x_feature[colname].factorize()

# MI method must have int type
discrete_features = train_x_feature.dtypes == int
print(discrete_features)


In [ ]:
from sklearn.feature_selection import mutual_info_classif
def make_MI_scores(x, y, features):
    sco = mutual_info_classif(x, y, discrete_features=features, random_state=123)
    sco = pd.Series(sco, name = "MI_Scores", index=x.columns)
    sco = sco.sort_values(ascending=False)
    
    return sco

MI_scores = make_MI_scores(train_x_feature, train_y_feature, discrete_features)
print(MI_scores)

In [ ]:
def fig_MI_scores(sco):
    sco = sco.sort_values(ascending=True)
    width = np.arange(len(sco))
    ticks = list(sco.index)
    
    plt.barh(width, sco)
    plt.yticks(width, ticks)
    plt.title('Mutual Information Scores')
    
plt.figure(dpi=100, facecolor='lightgray', figsize=(10,4))
fig_MI_scores(MI_scores.head(15))

In [ ]:
plt.figure(dpi=100, facecolor='lightgray', figsize=(10,4))
fig_MI_scores(MI_scores.tail(15))

## Model

### HistGradientBoostingClaasifier

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model1_val = HistGradientBoostingClassifier(max_iter=250,
                                         validation_fraction=None, 
                                         learning_rate=0.01, 
                                         max_depth=10, 
                                         min_samples_leaf=24, 
                                         max_leaf_nodes=60,
                                         random_state=123)

model1_val.fit(train_df_x, train_df_y)

In [ ]:
from sklearn.metrics import accuracy_score
pred1 = model1_val.predict(val_df_x)
acc1 = accuracy_score(val_df_y, pred1)
print('Accuracy: ', acc1)

In [ ]:
model1 = HistGradientBoostingClassifier(max_iter=250,
                                        validation_fraction=None, 
                                        learning_rate=0.01, 
                                        max_depth=10, 
                                        min_samples_leaf=24, 
                                        max_leaf_nodes=60,
                                        random_state=123,
                                        verbose=0)

model1.fit(train_x, train_y)
pred1 = model1.predict_proba(train_df_x_scaled)

**Submission**

In [ ]:
sub1 = sample_df.copy()

In [ ]:
sub1.iloc[:, 1:] = pred1.data
sub1.to_csv("submission1.csv",index=False)
display(sub1)